In [1]:
import os     

In [2]:
%pwd

'/Users/darebadejo/Documents/DS/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/darebadejo/Documents/DS/Text-Summarizer'

In [ ]:
# this is the entiity 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config

In [8]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self) -> Path:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(
                f"File: {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    
    def extract_zip_file(self) -> None:
        """
        zip_file_path : str 
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted to {unzip_path}")

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-07 23:10:31,119: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-11-07 23:10:31,121: INFO: common: YAML file: params.yaml loaded successfully]
[2025-11-07 23:10:31,122: INFO: common: Created directory at: artifacts]
[2025-11-07 23:10:32,324: INFO: 3928322216: File: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8746:3AC96C:19B346:1CB1A5:690EC150
Accept-Ranges: bytes
Date: Sat, 08 Nov 2025 04:10:32 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-klot8100129-CHI
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1762575032.011822,VS0,VE44
Vary: Authorizatio